In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.1 MB/s eta 0:00:00


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import os
from tqdm import tqdm
from glob import glob
import cv2
import numpy as np
import pandas as pd
import PIL 
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import torchvision as tv
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount._DEBUG = True
drive.mount('/content/drive', force_remount=True)

In [ ]:
os.chdir('./drive/MyDrive/이상치/')

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
    
device

In [ ]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

# 데이터 증강(data argumentation)

In [ ]:
# 하는 방법
import albumentations
import albumentations.pytorch

aug = albumentations.Compose([
      albumentations.Resize(224, 224),
      albumentations.HorizontalFlip(), #수평 뒤집기기 
      albumentations.VerticalFlip(), #수직 뒤집기
      albumentations.OneOf([
                          albumentations.Rotate(), # 돌리기기
                          albumentations.ShiftScaleRotate()
 
      ], p=1),
      albumentations.augmentations.transforms.Normalize(mean=(0.5,), std=(0.5,), p=1.0), #정규화화
      albumentations.pytorch.transforms.ToTensorV2(p=1.0)
      ])
aug2 = albumentations.Compose([
      albumentations.Resize(224, 224),
      albumentations.Rotate(), # 돌리기기
      albumentations.augmentations.transforms.Normalize(mean=(0.5,), std=(0.5,), p=1.0),
      albumentations.pytorch.transforms.ToTensorV2(p=1.0)
      ])

#  학습(다른 예시)

In [ ]:
# 시도 횟수
n_epochs = 100
valid_loss_min = np.inf # 100보다 작을 시 멈춤

# keep track of training and validation loss
train_loss = torch.zeros(n_epochs)
valid_loss = torch.zeros(n_epochs)

train_F1 = torch.zeros(n_epochs)
valid_F1 = torch.zeros(n_epochs)
model.to(device)

for e in range(0, n_epochs):

   
    ###################
    # 모델 학습       #
    ###################
    model.train()
    for data, labels in tqdm(train_dataloader):
        # move tensors to GPU if CUDA is available
        data, labels = data.to(device), labels.to(device)
        # 순전파 :compute predicted outputs by passing inputs to the model
        logits = model(data)
        # 배치의 손실율 계산
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        # 역전파 : compute gradient of the loss with respect to model parameters
        loss.backward()
        # 싱글 옵티미제이션(adam)
        optimizer.step()
        # 학습 손실율 업데이트
        train_loss[e] += loss.item()
        # 학습 스코어 계산
        logits=logits.argmax(1).detach().cpu().numpy().tolist()
        labels=labels.detach().cpu().numpy().tolist()

        train_F1[e] += score_function(labels,logits)

    train_loss[e] /= len(train_dataloader)
    train_F1[e] /= len(train_dataloader)
        
        
    ######################    
    # 검증 모델          #
    ######################
    with torch.no_grad(): 
        model.eval()
        for data, labels in tqdm(valid_dataloader):
            # move tensors to GPU if CUDA is available
            data, labels = data.to(device), labels.to(device)
            # 순전파: compute predicted outputs by passing inputs to the model
            logits = model(data)
            # 배치 손실율 계산
            loss = criterion(logits, labels)
            # 평균 검증 손실율 
            valid_loss[e] += loss.item()
            # update training score
            logits=logits.argmax(1).detach().cpu().numpy().tolist()
            labels=labels.detach().cpu().numpy().tolist()
            valid_F1[e] += score_function(labels,logits)
            
    
    # 평균 손실율 계산산
    valid_loss[e] /= len(valid_dataloader)
    valid_F1[e] /= len(valid_dataloader)
    
    scheduler.step(valid_loss[e])    
    # 학습/검증 결과 산출 표시(loss)
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        e, train_loss[e], valid_loss[e]))
    
    # 학습/검증 결과 산출 표시(정확도도)
    print('Epoch: {} \tTraining accuracy: {:.6f} \tValidation accuracy: {:.6f}'.format(
        e, train_F1[e], valid_F1[e]))
    
    # 검증 손실율이 줄어들었을 때 모델 저장장
    if valid_loss[e] <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss[e]))
        torch.save(model.state_dict(), 'swin_tiny_patch4_window7_224.pt')
        valid_loss_min = valid_loss[e]

# 모델 로드

In [ ]:
model.load_state_dict(torch.load('swin_tiny_patch4_window7_224.pt'))

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, x_dir,transform=None):
        super().__init__()
        self.transforms = transform
        self.x_img = x_dir 

    def __len__(self):
        return len(self.x_img)

    def __getitem__(self, idx):
        x_img = self.x_img[idx]

        x_img = cv2.imread(x_img)
        x_img = cv2.cvtColor(x_img, cv2.COLOR_BGR2RGB)

        if self.transforms:
            augmented = self.transforms(image=x_img)
            x_img = augmented['image']

        return x_img

In [ ]:
means=(0.5,)
stds=(0.5,)
testtransform = albumentations.Compose([
      albumentations.Resize(224, 224),
      albumentations.augmentations.transforms.Normalize(mean=means, std=stds, p=1.0),
      albumentations.pytorch.transforms.ToTensorV2(p=1.0)
      ])

In [ ]:
batch_size=64
test_dataset = TestDataset(test,testtransform)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# 추론(TTA)

# 모델 앙상블

In [ ]:
 def transform_inputsize(inputsize):
  return albumentations.Compose([
      albumentations.Resize(inputsize, inputsize),
      albumentations.augmentations.transforms.Normalize(mean=(0.5,), std=(0.5,), p=1.0),
      albumentations.pytorch.transforms.ToTensorV2(p=1.0)
      ])

In [ ]:
batch_size=32
test_dataset224 = TestDataset(test,transform_inputsize(224))
test_loader224 = torch.utils.data.DataLoader(test_dataset224, shuffle=False, batch_size=batch_size)
test_dataset300 = TestDataset(test,transform_inputsize(300))
test_loader300 = torch.utils.data.DataLoader(test_dataset300, shuffle=False, batch_size=batch_size)

모델 로드

In [ ]:
swin_tiny_patch4_window7_224 = timm.create_model('swin_tiny_patch4_window7_224',pretrained=True,num_classes=88,in_chans=3)
swin_tiny_patch4_window7_224.load_state_dict(torch.load('swin_tiny_patch4_window7_224.pt'))
mixnet_s=timm.create_model('mixnet_s',pretrained=True,num_classes=88,in_chans=3)
mixnet_s.load_state_dict(torch.load('mixnet_s.pt'))

In [ ]:
efficientnetB2 = EfficientNet.from_pretrained('efficientnet-b2', num_classes=88)
efficientnetB2.load_state_dict(torch.load('efficientnet-b2.pt'))
efficientnetB0 = EfficientNet.from_pretrained('efficientnet-b0', num_classes=88)
efficientnetB0.load_state_dict(torch.load('efficientnet-b0.pt'))

In [ ]:
tta_swin_tiny_patch4_window7_224 = tta.ClassificationTTAWrapper(swin_tiny_patch4_window7_224, tta_transforms)
tta_mixnet_s = tta.ClassificationTTAWrapper(mixnet_s, tta_transforms) 
tta_efficientnetB2 = tta.ClassificationTTAWrapper(efficientnetB2, tta_transforms) 
tta_efficientnetB0 = tta.ClassificationTTAWrapper(efficientnetB0, tta_transforms) 

추론

In [ ]:
tta_swin_tiny_patch4_window7_224.to(device)
tta_swin_tiny_patch4_window7_224.eval()
tta_mixnet_s.to(device)
tta_mixnet_s.eval()
tta_efficientnetB2.to(device)
tta_efficientnetB2.eval()
tta_efficientnetB0.to(device)
tta_efficientnetB0.eval()
f_pred = []

with torch.no_grad():
    for img224,img300 in iter(zip(test_loader224,test_loader300)):

        img224 = img224.to(device)
        img300 = img300.to(device)

        pred1 = tta_swin_tiny_patch4_window7_224(img224)
        pred2 = tta_efficientnetB2(img300)
        pred3 = tta_efficientnetB0(img300)
        pred4 = tta_mixnet_s(img300)

        pred = (pred1+pred2+pred3+pred4)/4
        
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())